# Project 3 - Net interest income prediction
- Description: Base on the Bank's nature, it's history performance and the previous year Balance Sheet, this year planning disbursement (in this prediction, ENR is used instead), this project will provide the estimation of Net interest income (NII) of the year.
- Project's value: This project helps BOM in planning the Targetted Disbursement volumes (here ENR instead) of the year so as to achieve the NII, as well as Profit
- Data source: base on public Financial Statements of the banks

## 0. Data loading & Description

In [89]:
import pandas as pd
import numpy as np
# Data loading and droping missing value
df = pd.read_csv('Raw.csv',sep= ',').dropna()
df


Bank    FY  Cur_TA_ENR   Pre_Total_Asset   Pre_TA_ENR   Pre_Liability   \
0   SCB  2015     $169.23           $242.22      $133.28         $229.04    
1   SCB  2016     $220.07           $311.51      $169.23         $296.06    
2   SCB  2017     $264.15           $361.68      $220.07         $346.22    
3   SCB  2018     $299.17           $444.03      $264.15         $428.50    
4   SCB  2019     $330.94           $508.95      $299.17         $492.38    
..  ...   ...          ...               ...          ...             ...   
90  BVB  2018      $29.35            $39.90       $24.78          $36.56    
91  BVB  2019      $33.54            $46.55       $29.35          $43.11    
92  BVB  2020      $39.26            $51.81       $33.54          $48.07    
93  BVB  2021      $45.69            $61.10       $39.26          $57.21    
94  BVB  2022      $50.12            $76.51       $45.69          $71.87    

    Pre_Equity   Pre_Retain_earning   Crisis  Structure_change  Top10  \
0       $13.19                $0.46        0                 0      0   
1       $15.24                $0.50        0                 0      0   
2       $15.46                $0.53        0                 0      0   
3       $15.37                $0.59        0                 0      0   
4       $16.42                $0.67        0                 0      0   
..          ...                  ...     ...               ...    ...   
90       $3.34                $0.19        0                 0      0   
91       $3.44                $0.28        0                 0      0   
92       $3.73                $0.39        1                 0      0   
93       $3.89                $0.53        1                 0      0   
94       $4.64                $0.75        1                 0      0   

    Domestic    NII   
0          1  $4.51   
1          1  $2.93   
2          1  $1.89   
3          1  $2.91   
4          1  $4.03   
..       ...     ...  
90         1  $0.80   
91         1  $0.93   
92         1  $1.10   
93         1  $1.43   
94         1  $1.71   

[95 rows x 13 columns]

In [90]:
# Xử lý khoản trắng và các ký tự đặc biệt
df.columns = df.columns.str.strip()
df = df.replace({'\$': ''}, regex=True)
df

Bank    FY Cur_TA_ENR Pre_Total_Asset Pre_TA_ENR Pre_Liability Pre_Equity  \
0   SCB  2015    169.23          242.22     133.28        229.04      13.19    
1   SCB  2016    220.07          311.51     169.23        296.06      15.24    
2   SCB  2017    264.15          361.68     220.07        346.22      15.46    
3   SCB  2018    299.17          444.03     264.15        428.50      15.37    
4   SCB  2019    330.94          508.95     299.17        492.38      16.42    
..  ...   ...        ...             ...        ...           ...        ...   
90  BVB  2018     29.35           39.90      24.78         36.56       3.34    
91  BVB  2019     33.54           46.55      29.35         43.11       3.44    
92  BVB  2020     39.26           51.81      33.54         48.07       3.73    
93  BVB  2021     45.69           61.10      39.26         57.21       3.89    
94  BVB  2022     50.12           76.51      45.69         71.87       4.64    

   Pre_Retain_earning  Crisis  Structure_change  Top10  Domestic    NII  
0               0.46        0                 0      0         1  4.51   
1               0.50        0                 0      0         1  2.93   
2               0.53        0                 0      0         1  1.89   
3               0.59        0                 0      0         1  2.91   
4               0.67        0                 0      0         1  4.03   
..                ...     ...               ...    ...       ...    ...  
90              0.19        0                 0      0         1  0.80   
91              0.28        0                 0      0         1  0.93   
92              0.39        1                 0      0         1  1.10   
93              0.53        1                 0      0         1  1.43   
94              0.75        1                 0      0         1  1.71   

[95 rows x 13 columns]

In [95]:
# Chuyển đổi string thành dạng float
cols = ['Cur_TA_ENR', 'Pre_Total_Asset', 'Pre_TA_ENR', 'Pre_Liability', 'Pre_Equity', 'Pre_Retain_earning', 'NII']
df[cols] = df[cols].astype(float)
df

Bank    FY  Cur_TA_ENR  Pre_Total_Asset  Pre_TA_ENR  Pre_Liability  \
0    NaN  2015      169.23           242.22      133.28         229.04   
1    NaN  2016      220.07           311.51      169.23         296.06   
2    NaN  2017      264.15           361.68      220.07         346.22   
3    NaN  2018      299.17           444.03      264.15         428.50   
4    NaN  2019      330.94           508.95      299.17         492.38   
..   ...   ...         ...              ...         ...            ...   
90   NaN  2018       29.35            39.90       24.78          36.56   
91   NaN  2019       33.54            46.55       29.35          43.11   
92   NaN  2020       39.26            51.81       33.54          48.07   
93   NaN  2021       45.69            61.10       39.26          57.21   
94   NaN  2022       50.12            76.51       45.69          71.87   

    Pre_Equity  Pre_Retain_earning  Crisis  Structure_change  Top10  Domestic  \
0        13.19                0.46       0                 0      0         1   
1        15.24                0.50       0                 0      0         1   
2        15.46                0.53       0                 0      0         1   
3        15.37                0.59       0                 0      0         1   
4        16.42                0.67       0                 0      0         1   
..         ...                 ...     ...               ...    ...       ...   
90        3.34                0.19       0                 0      0         1   
91        3.44                0.28       0                 0      0         1   
92        3.73                0.39       1                 0      0         1   
93        3.89                0.53       1                 0      0         1   
94        4.64                0.75       1                 0      0         1   

     NII  
0   4.51  
1   2.93  
2   1.89  
3   2.91  
4   4.03  
..   ...  
90  0.80  
91  0.93  
92  1.10  
93  1.43  
94  1.71  

[95 rows x 13 columns]

Cần một số phân tích ở đây

## 1. Data preparation

In [96]:
# Data splitting
y = df['NII']
x = df.drop(['NII','Bank'], axis=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train , y_test = train_test_split(x,y, train_size=0.8, random_state=0)

## 2. Training Models

In [97]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state=1)

dt_model.fit(x_train,y_train)
y_train_pre = dt_model.predict(x_train)
y_test_pre = dt_model.predict(x_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## 3. Model Evaluation

In [98]:
from sklearn.metrics import mean_squared_error, r2_score

lr_train_mse = mean_squared_error(y_train, y_train_pre)
lr_train_r2 = r2_score(y_train, y_train_pre)

lr_test_mse = mean_squared_error(y_test, y_test_pre)
lr_test_r2 = r2_score(y_test, y_test_pre)

NameError: name 'y_lr_train_pred' is not defined

Insights get: ...